In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

RAW  = Path("../data/raw")
PROC = Path("../data/processed"); PROC.mkdir(parents=True, exist_ok=True)

peers_raw = pd.read_csv(RAW / "peers" / "all_peers_fnltt_raw_long.csv", dtype=str)

def _to_num_million_won(x):
    if pd.isna(x): return np.nan
    s = str(x).replace(",", "").replace("(", "-").replace(")", "").strip()
    return pd.to_numeric(s, errors="coerce") / 1e8

keep = ["peer_stock_code","peer_corp_name","bsns_year","sj_div","account_nm",
        "thstrm_amount","frmtrm_amount"]
p = peers_raw[keep].copy()
p["thstrm_amount"] = p["thstrm_amount"].map(_to_num_million_won)
p["frmtrm_amount"] = p["frmtrm_amount"].map(_to_num_million_won)

p = p.rename(columns={"peer_stock_code":"ticker","peer_corp_name":"company",
                      "bsns_year":"year","sj_div":"statement"})
p["year"] = p["year"].astype(int)

cur = p.rename(columns={"thstrm_amount":"amount"})[["ticker","company","year","statement","account_nm","amount"]]
pri = p.rename(columns={"frmtrm_amount":"amount"})[["ticker","company","year","statement","account_nm","amount"]]
pri["year"] = pri["year"] - 1

peer_long = (pd.concat([cur, pri], ignore_index=True)
               .dropna(subset=["amount"])
               .sort_values(["company","year"]))
peer_long.to_csv(PROC / "peer_long.csv", index=False)
peer_long.head(3)


,ticker,company,year,statement,account_nm,amount
12693,051900,LG생활건강,2019,BS,유동자산,21630.748410
12694,051900,LG생활건강,2019,BS,현금및현금성자산,6471.040592
12695,051900,LG생활건강,2019,BS,단기금융상품,722.634506


In [ ]:
def _read_jion_sheet(xlsx_path, sj):
    df = pd.read_excel(xlsx_path)
    def _pick(cols, pats, default_idx):
        for c in cols:
            if any(p in str(c) for p in pats): return c
        return cols[default_idx]
    c_acc = _pick(df.columns, ["계정","항목","과목","account"], 0)
    c_cur = _pick(df.columns, ["당기","current","thstrm"], 1)
    c_pri = _pick(df.columns, ["전기","prior","previous","frmtrm"], 2)

    out = df[[c_acc, c_cur, c_pri]].copy()
    out.columns = ["account_nm","thstrm_amount","frmtrm_amount"]
    for c in ["thstrm_amount","frmtrm_amount"]:
        out[c] = (out[c].astype(str)
                        .str.replace(",", "", regex=False)
                        .str.replace("(", "-", regex=False)
                        .str.replace(")", "", regex=False))
        out[c] = pd.to_numeric(out[c], errors="coerce")/1e8
    out = out.dropna(subset=["account_nm"]).assign(statement=sj)
    return out

JRAW = RAW / "jion"
is_df = _read_jion_sheet(JRAW / "Jion IS.xlsx", "IS")
bs_df = _read_jion_sheet(JRAW / "Jion BS.xlsx", "BS")
cf_df = _read_jion_sheet(JRAW / "Jion CF.xlsx", "CF")

j_all = pd.concat([is_df, bs_df, cf_df], ignore_index=True)

CUR_YEAR = 2025
j_cur = j_all[["statement","account_nm","thstrm_amount"]].rename(columns={"thstrm_amount":"amount"}).assign(company="지온메디텍", ticker=None, year=CUR_YEAR)
j_pri = j_all[["statement","account_nm","frmtrm_amount"]].rename(columns={"frmtrm_amount":"amount"}).assign(company="지온메디텍", ticker=None, year=CUR_YEAR-1)

jion_long = pd.concat([j_cur, j_pri], ignore_index=True).dropna(subset=["amount"]).sort_values("year")
jion_long.to_csv(PROC / "jion_long.csv", index=False)
jion_long.head(3)


,statement,account_nm,amount,company,ticker,year
362,CF,V. 기초의 현금,49.674688,지온메디텍,None,2024
361,CF,IV. 현금의 증가(감소)(I+II+III),15.355105,지온메디텍,None,2024
321,CF,부가세대급금의 감소(증가),-11.728670,지온메디텍,None,2024


In [3]:
import re, unicodedata

def _norm(s):
    s = unicodedata.normalize("NFKC", str(s)).lower()
    return re.sub(r"[^\w가-힣]+", "", s)

SYN = {
    "revenue":  ["매출액","매출","영업수익","revenue","sales","salesrevenue"],
    "cogs":     ["매출원가","원가","cogs","costofsales","costofgoodsold"],
    "op_inc":   ["영업이익","operatingincome","operatingprofit"],
    "ad_exp":   ["광고선전비","advertisingexpense","advertising","marketingexpense"],
    "inventories": ["재고자산","inventories","inventory"],
    "inv_val_loss": ["재고자산평가손실","inventoryvaluationloss","lossonvaluationofinventories"],
    "cur_assets":   ["유동자산","currentassets"],
    "cur_liab":     ["유동부채","currentliabilities"],
    "tot_liab":     ["부채총계","totalliabilities","liabilitiestotal"],
    "equity":       ["자본총계","totalequity","equitytotal","stockholdersequity"],
    "cfo":          ["영업활동으로인한현금흐름","영업활동현금흐름","cashflowsfromoperatingactivities",
                     "netcashprovidedbyoperatingactivities","operatingcashflows"],
    "inc_st_borrow": ["단기차입금의증가","increaseinshorttermborrowings","shorttermborrowingsincrease"]
}
SYN_N = {k: {_norm(x) for x in v} for k, v in SYN.items()}

def _canon(acct):
    nm = _norm(acct)
    for k, keys in SYN_N.items():
        if any(key in nm for key in keys) or nm in keys:
            return k
    return None

def to_canon_wide(df):
    d = df.copy()
    d["canon"] = d["account_nm"].map(_canon)
    d = d[d["canon"].notna()]
    wide = (d.pivot_table(index=["company","ticker","year"], columns="canon", values="amount", aggfunc="first")
              .reset_index()
              .sort_values(["company","year"]))
    return wide

peer_wide = to_canon_wide(peer_long)
jion_wide = to_canon_wide(jion_long)
peer_wide.to_csv(PROC / "peer_wide.csv", index=False)
jion_wide.to_csv(PROC / "jion_wide.csv", index=False)
peer_wide.tail(2), jion_wide


(canon company  ticker  year  ad_exp          cfo  cogs   cur_assets  \
 16       에이피알  278470  2023     NaN  1078.409614   NaN  2193.079957   
 17       에이피알  278470  2024     NaN   791.241617   NaN  2860.274454   
 
 canon     cur_liab       equity  inc_st_borrow  inventories       op_inc  \
 16      822.964214  1969.488688            NaN   565.209724  1041.937249   
 17     1451.367586  3235.240740            NaN  1097.456770  1227.055444   
 
 canon     revenue     tot_liab  
 16     230.052217   903.606195  
 17     479.474061  2416.194892  ,
 Empty DataFrame
 Columns: [company, ticker, year]
 Index: [])

In [5]:
import pandas as pd
import numpy as np

def build_metrics(wide: pd.DataFrame) -> pd.DataFrame:
    out = wide.copy()
    out = out.sort_values(["company","year"]).reset_index(drop=True)
    eps = 1e-9

    def col(colname, fill=np.nan):
        # return a Series aligned to out if missing
        if colname in out:
            s = pd.to_numeric(out[colname], errors="coerce")
            return s
        return pd.Series(fill, index=out.index, dtype="float64")

    revenue   = col("revenue")
    cogs      = col("cogs")
    op_inc    = col("op_inc")
    ad_exp    = col("ad_exp")
    inv       = col("inventories")
    inv_loss  = col("inv_val_loss")
    cur_ast   = col("cur_assets")
    cur_liab  = col("cur_liab")
    tot_liab  = col("tot_liab")
    equity    = col("equity")
    cfo       = col("cfo")
    inc_st    = col("inc_st_borrow", fill=0.0)  # treat missing as 0 for the flag

    # margins & spend ratios
    out["gross_margin"]   = (revenue - cogs) / (revenue + eps)
    out["op_margin"]      =  op_inc / (revenue + eps)
    out["ad_to_sales"]    =  ad_exp / (revenue + eps)
    out["inv_to_sales"]   =  inv / (revenue + eps)
    out["cfo_margin"]     =  cfo / (revenue + eps)

    # BS ratios
    out["current_ratio"]  =  cur_ast / (cur_liab + eps)     # current ratio
    out["debt_to_equity"] =  tot_liab / (equity + eps)      # debt/equity

    # changes
    out["cfo_yoy"]        = out.groupby("company")["cfo"].pct_change() if "cfo" in out else np.nan


    out["flag_inv_val_loss"]  = inv_loss.notna() & (inv_loss > 0)
    out["flag_inc_st_borrow"] = inc_st.notna()  & (inc_st  > 0)

    return out

peer_metrics = build_metrics(peer_wide)
jion_metrics = build_metrics(jion_wide)

PROC.mkdir(parents=True, exist_ok=True)
peer_metrics.to_csv(PROC / "metrics_peers.csv", index=False)
jion_metrics.to_csv(PROC / "metrics_jion.csv", index=False)